# DETROIT CRIME ANALYSIS
##### This project focuses on developing a **time-series forecasting** system to optimize public safety strategies and resource allocation. By building an **end-to-end data pipeline**, the project aggregates and **geocodes** large-scale crime incident reports to identify **spatial-temporal patterns**.

##### The core technical effort involves **training and benchmarking statistical models (SARIMA, Prophet)** against advanced **deep learning architectures (LSTM, BLSTM)** to determine the most accurate predictors of criminal activity. These models are utilized to generate long-term forecasts, specifically pinpointing high-risk districts. The final output transforms raw data into actionable intelligence, enabling data-driven decisions for effective police deployment and crime prevention.

## PART 1 - Data Acquisition & Initial Exploration

##### The Datasets are downloaded from [Detroit Open Data Portal](https://data.detroitmi.gov/datasets "Detroit Open Data Portal").
##### 1. This initial phase focused on architecting a data ingestion to support downstream predictive modeling. The objective was to synthesize heterogeneous datasets to investigate the relationship between public safety and licensed establishments.
##### 2. Extraction of [RMS Crime Incidents](https://data.detroitmi.gov/datasets/detroitmi::rms-crime-incidents/explore?location=42.347872%2C-83.095882%2C10.53) from the Detroit Open Data API (2016–Present) and State of Michigan [Liquor License records](https://data.detroitmi.gov/datasets/detroitmi::liquor-licenses-current/explore?location=42.302893%2C-83.018479%2C10.20).
##### 3. To ensure analytical reliability, conducted a rigorous Exploratory Data Analysis (EDA) and data quality assessment. Leveraging pandas and numpy for manipulation and matplotlib for visualization, generated comprehensive data quality reports. These audits—ranging from missing value trends over time to temporal coverage timelines—established a verified, clean baseline for the project, eliminating data inconsistencies prior to the modeling stage.